Import all packages

In [ ]:
import numpy as np 
import sep 
from astropy.io import fits 
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.utils.data import get_pkg_data_filename
from matplotlib.patches import Ellipse

Tell Astropy where to find .Fits file on my mac

In [ ]:
image_file = get_pkg_data_filename('/Users/aryaazizi/Desktop/UCSC/Astr_119/Final/image.fits')

In [ ]:
fits.info(image_file)   #to better understand file data 

Read data into a 2d numpy array 

In [ ]:
data = fits.getdata(image_file)

In [ ]:
print(data.shape)
print(type(data))      #more info on data 

Plot the numpy data with matplotlib, and save the image  

In [ ]:
plt.figure()
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

plt.show()
plt.savefig('displayed_Fits_file.png')

Measure varying background data 

In [ ]:
bkg = sep.Background(data)
bkg = sep.Background(data, mask=None, bw=64, bh=64, fw=3, fh=3)

get mean and noise of background data 

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

evaluate background as 2-d array

In [ ]:
bkg_image = bkg.back()

Show plot and save image 

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.show()
plt.savefig('Background.png')

In [ ]:
Subtract background data 

In [ ]:
data_sub = data - bkg

Set detection threshold for object detection 

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Print objects in list 

In [ ]:
print(len(objects))

In [ ]:
# plot background data
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot ellipse for each object 
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('green')
    ax.add_artist(e)
objects.dtype.names
plt.show()
plt.savefig('Object_Detection.png')

circular aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

Show first 10 object results 

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

Plot Histogram of fluxes

In [ ]:
fig = plt.subplots(figsize =(6, 6))
plt.hist(flux, bins = 5)
plt.show()
plt.savefig('Histogram.png')